In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import numpy as np
from qiskit_ibm_runtime import Session, Estimator, QiskitRuntimeService, Options
from qiskit.quantum_info import Operator, Statevector

from nonlocalgames.hamiltonians import G14
from nonlocalgames.circuit import NLGCircuit, load_adapt_ansatz
from nonlocalgames.measurement import MeasurementLayer

2023-09-11 16:16:49,786	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [17]:
def get_pcc() -> Operator:
    proj = 0
    for c in range(4):
        s = f'{c:02b}'[::-1]
        s = s + s
        cc = Statevector.from_label(s)
        pcc = cc.to_operator()
        proj += pcc
    
    return proj

def load_g14_circuit(path: str):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    qubits = 2
    players = 2
    qc = load_adapt_ansatz(
        data['state'],
        '++++',
        [qubits] * players,
        adapt_order=False,
        min_theta = 1e-4)

    # Don't need to transform phi since qinfo's Ry gate follows the qiskit convention
    phi = np.array(data['phi'])

    constrained = False
    if 'constrained' in path:
        constrained = True

    if 'metadata' in data:
        layer = data['metadata'].get('layer', 'ry')
        constrained = data['metadata'].get('constrain_phi', constrained)
    else:
        layer = 'ry'
    
    ml = MeasurementLayer.get(layer, players=2, questions=14, qubits=2)
    
    if constrained:
        if phi.ndim == 1:
            phi = np.concatenate([phi, phi]).reshape(ml.shape)
        else:
            phi = np.concatenate([phi, phi])
            assert phi.shape == ml.shape
    
    ml.params = phi

    if constrained:
        ml.conj(1)

    return qc, ml

In [28]:
qc, ml = load_g14_circuit('../../data/g14_constrained_u3ry/g14_state.json')
nlg = NLGCircuit(qc, None, measurement_layer=ml)

vertices = [(v, v) for v in range(14)]
edges = G14._get_graph().edge_links.tolist()
questions = vertices + edges

pcc = get_pcc()
pcc.dim

(16, 16)

In [31]:
options = Options()
options.execution.shots = 1024
options.optimization_level = 3
options.resilience_level = 2  # ZNE
options.resilience.noise_factors = [1, 2, 3, 4]
# options.resilience.noise_amplifier = "LocalFoldingAmplifier"

service = QiskitRuntimeService(name='full')
backend = service.get_backend('ibm_hanoi')
# backend = service.get_backend('simulator_statevector')
with Session(service=service, backend=backend) as session:
    nlg.backend = backend
    estimator = Estimator(session=session, options=options)
    circuits = list(map(nlg._prepare_question, questions))
    for circuit in circuits:
        circuit.remove_final_measurements()
    observables = [pcc] * len(circuits)

    job = estimator.run(circuits=circuits, observables=observables)
    print('Job:', job.job_id())
    result = job.result()

Job: ck09h6siifjafcscd6b0


In [38]:
path = 'zne_results_optim{optim}_zne{zne}.json'.format(
            optim=options.optimization_level,
            zne=options.resilience_level
        )
metadata = result.metadata
values = result.values.tolist()
for m, q in zip(metadata, questions):
    m['question'] = q
with open(path, 'w', encoding='utf-8') as f:
    json.dump({
        'metadata': result.metadata,
        'values': result.values.tolist()
    }, f)

In [40]:
import pandas as pd

records = []

for m, zne_value in zip(metadata, values):
    zne_data = m['zne']['noise_amplification']
    for noise_level, obs_value, obs_var in zip(
        zne_data['noise_factors'],
        zne_data['values'],
        zne_data['variance']):

        records.append({
            'va': m['question'][0],
            'vb': m['question'][1],
            'noise_factor': noise_level,
            'obs_value': obs_value,
            'obs_var': obs_var,
            'extrapolated_value': zne_value
        })

df = pd.DataFrame.from_records(records)
path = 'zne_results_optim{optim}_zne{zne}.csv'.format(
            optim=options.optimization_level,
            zne=options.resilience_level
        )
df.sort_values(['va', 'vb'], inplace=True)
df.to_csv(path, index=False)
df

,va,vb,noise_factor,obs_value,obs_var,extrapolated_value
0,0,0,1,0.948242,0.024831,0.958008
1,0,0,2,0.972656,0.013374,0.958008
2,0,0,3,0.977539,0.011040,0.958008
3,0,0,4,0.953125,0.022566,0.958008
56,0,1,1,0.956055,0.021234,0.953125
...,...,...,...,...,...,...
203,13,12,4,0.250000,0.187441,0.947266
52,13,13,1,0.963867,0.017573,0.929687
53,13,13,2,0.875977,0.055984,0.929687
54,13,13,3,0.896484,0.046858,0.929687
